In [1]:
import polars as pl

In [2]:
jobs = pl.read_csv("../data/unique_jobs.csv").select(["INDEX", "UNIQUE JOBS"])

cats = pl.read_json("../data/categorized_data.json")

agg = pl.read_csv("../data/WildRP_job_data_values_only.csv")

raw_data = pl.read_csv("../data/WildRP Job Data - Raw Data.csv")

value_data = pl.read_csv("../data/WildRP_job_data_values_only.csv").rename({"": "JOB"} )

In [3]:
value_data = value_data.filter(pl.col("JOB").is_not_null())

In [6]:
value_data.head(1)

JOB,JobType,JobTemplateId,Abandon Count,Completion Count,Failed Count,Fatal Error Count,_duplicated_0,CharacterId,Abandon Count_duplicated_0,Completion Count_duplicated_0
str,str,i64,i64,i64,i64,i64,str,i64,i64,i64
"""Bring a Crate of 5 Bank Bonds …","""Delivery""",751,3,0,0,0,null,14320,12,58


In [14]:
full_jobs = value_data.select(pl.col(["JobTemplateId", "JOB"])).join(
    raw_data,
    on="JobTemplateId",
    how="inner")

In [22]:
full_jobs.head(1)

JobTemplateId,JOB,JobStarted,JobEnded,TimeLeft,JobState,ErrorMessage,LeadingCharacterId,SerializedCharacterIds,LawInformed,JobType,CharacterId,JobType_duplicated_0,SerializedDeliveriesCompleted,SerializedRanchChoreJobStatus,SerializedDestinations,SerializedHuntingJobStatus,DepositCompleted,HerdLost,HerdedCount,HerdingState,RetryCount,SerializedNetworkIds
i64,str,str,str,i64,i64,str,i64,str,i64,str,i64,str,str,str,str,str,i64,i64,i64,i64,i64,str
613,"""Crates with 30 Coal each to be…","""2024-04-25 4:35:30""",null,0,3,null,0,null,0,"""Delivery""",10282,"""Delivery""",null,null,"""[""caliga_hall"",""hillhaven_ranc…",null,null,0,0,0,0,null


In [21]:
full_jobs.group_by(["CharacterId","JOB"]).len().sort("len", descending=True).head(10)


CharacterId,JOB,len
i64,str,u32
20719,"""Help out with chores at Prongh…",199
18633,"""Help out with chores at Prongh…",189
5544,"""Help out with chores at Prongh…",178
22556,"""Help with chores at Macfarlane…",148
10071,"""Help out with chores at Prongh…",113
20719,"""Travel to Downes Ranch and hel…",110
14796,"""Help out with chores at Prongh…",69
19433,"""Help out with chores at Emeral…",58
18675,"""Help out with chores at Prongh…",52


In [4]:
sample_df = (jobs.join(cats, on="INDEX")
 	 .drop("JOB")
	 .join(agg, left_on="UNIQUE JOBS", right_on="")
)

In [5]:
sample_df = sample_df.rename({"Completion Count": "COMPLETION_COUNT",
							  "Abandon Count": "ABANDON_COUNT",
							  })

In [6]:
# Calculate averages
avg_completion_count_wagon = sample_df.filter(pl.col("REQUIRES_WAGON") == 1)["COMPLETION_COUNT"].mean()
avg_completion_count_no_wagon = sample_df.filter(pl.col("REQUIRES_WAGON") == 0)["COMPLETION_COUNT"].mean()
avg_abandon_count_wagon = sample_df.filter(pl.col("REQUIRES_WAGON") == 1)["ABANDON_COUNT"].mean()
avg_abandon_count_no_wagon = sample_df.filter(pl.col("REQUIRES_WAGON") == 0)["ABANDON_COUNT"].mean()
# IS_CRIMINAL averages
avg_completion_count_criminal = sample_df.filter(pl.col("IS_CRIMINAL") == 1)["COMPLETION_COUNT"].mean()
avg_completion_count_not_criminal = sample_df.filter(pl.col("IS_CRIMINAL") == 0)["COMPLETION_COUNT"].mean()
avg_abandon_count_criminal = sample_df.filter(pl.col("IS_CRIMINAL") == 1)["ABANDON_COUNT"].mean()
avg_abandon_count_not_criminal = sample_df.filter(pl.col("IS_CRIMINAL") == 0)["ABANDON_COUNT"].mean()

# Create a DataFrame with the results
result_df = pl.DataFrame({
    "AVG_COMPLETION_COUNT_WAGON": [avg_completion_count_wagon],
    "AVG_COMPLETION_COUNT_NO_WAGON": [avg_completion_count_no_wagon],
    "AVG_ABANDON_COUNT_WAGON": [avg_abandon_count_wagon],
    "AVG_ABANDON_COUNT_NO_WAGON": [avg_abandon_count_no_wagon],
    "AVG_COMPLETION_COUNT_CRIMINAL": [avg_completion_count_criminal],
    "AVG_COMPLETION_COUNT_NOT_CRIMINAL": [avg_completion_count_not_criminal],
    "AVG_ABANDON_COUNT_CRIMINAL": [avg_abandon_count_criminal],
    "AVG_ABANDON_COUNT_NOT_CRIMINAL": [avg_abandon_count_not_criminal],
})



In [7]:
avg_completion_count_criminal / avg_abandon_count_criminal

3.1770833333333335

In [8]:
avg_completion_count_not_criminal / avg_abandon_count_not_criminal

2.1954161640530763

In [9]:
avg_completion_count_wagon /  avg_completion_count_no_wagon

1.136444229529336

In [10]:
result_df

AVG_COMPLETION_COUNT_WAGON,AVG_COMPLETION_COUNT_NO_WAGON,AVG_ABANDON_COUNT_WAGON,AVG_ABANDON_COUNT_NO_WAGON,AVG_COMPLETION_COUNT_CRIMINAL,AVG_COMPLETION_COUNT_NOT_CRIMINAL,AVG_ABANDON_COUNT_CRIMINAL,AVG_ABANDON_COUNT_NOT_CRIMINAL
f64,f64,f64,f64,f64,f64,f64,f64
5.335106,4.694561,2.180851,2.154812,7.261905,4.727273,2.285714,2.153247
